# 初期設定

Scrapyのインストール & プロジェクト作成

In [ ]:
%%bash
pip install scrapy
scrapy startproject tutorial
cd tutorial
scrapy genspider zozo zozo.jp

今後のscrapyコマンドの実行のために、カレントディレクトリを切り替えておく

In [ ]:
%cd /content/tutorial

settings.pyに初期設定を書き込む

In [18]:
%%writefile -a tutorial/settings.py

# User Agentを主要ブラウザと同じものに設定
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/602.4.8 (KHTML, like Gecko) Version/10.0.3 Safari/602.4.8'

# ダウンロード間隔を1秒に設定
DOWNLOAD_DELAY = 1

# パース結果の日本語を出力するために設定
FEED_EXPORT_ENCODING = 'utf-8'

Appending to tutorial/settings.py


# ZOZOTOWNの商品名を取得する

In [ ]:
%%writefile tutorial/spiders/zozo.py
import scrapy

class ZozoSpider(scrapy.Spider):
    # コマンドラインから起動する時の名前
    name = 'zozo'
    allowed_domains = ['zozo.jp']
    # 商品詳細URLを指定
    start_urls = [
			'https://zozo.jp/shop/zozo/goods-sale/52717046/'
    ]

    def parse(self, response):
        # 商品名を取得するためのXPathを指定
        title = response.xpath('//h1/text()').get().strip()
        yield {'title': title }


scrapy crawlコマンドでクローリングを実行

In [ ]:
%%bash
scrapy crawl zozo -o result.csv

In [ ]:
%%bash
cat result.csv

# ページをたどりながら複数商品をクローリングする

In [ ]:
%%writefile -a tutorial/spiders/zozo.py

    # クローリング処理の開始
    def start_requests(self):
        url = 'https://zozo.jp/category/tops/'
        yield scrapy.Request(url=url, callback=self.parse_list)

    def parse_list(self, response):
        # 次のリストページをたどる処理
        next_page_url = response.xpath("次ページURLのXPath").get()
        yield response.follow(next_page_url, callback=self.parse_list)

        # 詳細ページをたどる処理
        detail_page_urls = response.xpath("詳細ページURLのXPath").getall()
        for detail_page_url in detail_page_urls:
            yield response.follow(detail_page_url, callback=self.parse)
